In [12]:
def cast_to_string(routes_property):
    return [str(int).replace("[", "").replace("]", "") for int in routes_property]

In [13]:
def flat_list(data):
    return [item for sublist in data for item in sublist]

In [43]:
graph = pd.read_csv("omsk_graph.csv", index_col = 0).reset_index().drop(["city","alt_road"], axis = 1).drop(["quartal_id", "zlevel_from", "zlevel_to","truck_zone_id", "ts", "version", "year_month", "fiji_id","seg_begin","seg_end","tile_from","tile_to", "azimuth_begin", "azimuth_end","street","geom","sign_segment_begin","sign_segment_end"], axis=1)

In [44]:
graph

,edge_id,node_from,node_to,length,width,road_class,sm_class,rm_car_class,rm_car_speed,type,light_begin,light_end,turn_to,payments_flags,is_city_edge,barrier,turn_restrictions,traverse,passability,pedo_offset,bad_road,style,lanes,def_speed,road_exit
0,564384472498178,8.735719e+16,8.735345e+16,242,4,3,10,7,60,0,0,0,1,0,1,0,0,1,1,1,0,7,1,60,0
1,564384472498179,8.735345e+16,8.735719e+16,242,4,3,10,7,60,0,0,0,1,0,1,0,0,1,1,1,0,7,1,60,0
2,564384472498182,8.735998e+16,8.735719e+16,224,4,3,6,7,60,0,0,0,1,0,1,0,0,1,1,1,0,7,1,60,0
3,564384472498183,8.735719e+16,8.735998e+16,224,4,3,10,7,60,0,0,0,1,0,1,0,0,1,1,1,0,7,1,60,0
4,564384472498184,8.755692e+16,8.755998e+16,444,4,3,12,9,90,0,0,0,1,0,0,0,0,1,1,1,0,7,1,90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231683,212117746323998779,8.758620e+16,8.758630e+16,7,3,9,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
231684,212117746366813904,8.769170e+16,8.769191e+16,11,3,9,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
231685,212117746366813905,8.769191e+16,8.769170e+16,11,3,9,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
231686,212117746395564138,8.773269e+16,8.773267e+16,16,3,9,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0


In [45]:
adj = []
adj_values = []
for i in tqdm(range(len(graph))):
    a = graph[graph["node_to"] == graph[graph["edge_id"] == graph.iloc[i, 0]]["node_from"][i]]["edge_id"].index
    b = list(graph[graph["node_from"] == graph[graph["edge_id"] == graph.iloc[i, 0]]["node_from"][i]]["edge_id"].index)
    b.remove(i)
    df = list(a) + b
    adj.append(df)
adj_str = cast_to_string(adj)
adj_df = pd.DataFrame(adj_str, columns = ["adjacent"])

100%|█████████████████████████████████████████████████████████████████████████| 231688/231688 [15:35<00:00, 247.56it/s]


In [9]:
adj_str = cast_to_string(adj)
adj_df = pd.DataFrame(adj_str, columns = ["adjacent"])

In [38]:
adj_df

,adjacent
0,"1, 2, 111230, 3, 111231"
1,"0, 111308, 111309"
2,"3, 6, 29, 7, 28"
3,"1, 2, 111230, 0, 111231"
4,"5, 16, 17"
...,...
231683,231682
231684,231685
231685,231684
231686,231687


In [39]:
adj_str = cast_to_string(adj)
adj_df = pd.DataFrame(adj_str, columns = ["adjacent"])

In [40]:
graph = graph.join(adj_df).drop(["node_from", "node_to"], axis = 1)

In [13]:
graph[["edge_id", "adjacent"]].to_csv("adj_list_omsk_readable.csv", index=False)

In [41]:
road_classes = pd.get_dummies(graph.road_class, prefix='road_classes')
sm_classes = pd.get_dummies(graph.sm_class, prefix='sm_classes')
rm_car_classes = pd.get_dummies(graph.rm_car_class, prefix='rm_car_classes')
styles = pd.get_dummies(graph.style, prefix='style')
sm_lanes = pd.get_dummies(graph.lanes, prefix='lanes')
graph_enc = graph.join(road_classes).join(sm_classes).join(styles).join(sm_lanes).join(rm_car_classes).drop(["road_class", "sm_class", "lanes", "style", "rm_car_class"], axis = 1)
graph_enc.drop(["edge_id", "adjacent"], axis = 1).to_numpy()

array([[242.,   4.,  60., ...,   0.,   0.,   0.],
       [242.,   4.,  60., ...,   0.,   0.,   0.],
       [224.,   4.,  60., ...,   0.,   0.,   0.],
       ...,
       [ 11.,   3.,   3., ...,   0.,   0.,   0.],
       [ 16.,   3.,   3., ...,   0.,   0.,   0.],
       [ 16.,   3.,   3., ...,   0.,   0.,   0.]])

In [42]:
graph_enc.to_csv("graph_omsk_F_encoded.csv")

Все что ниже перенесено в ноутбуки с конкретными моделями (т. е. не представляет значимости)

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

from tqdm import tqdm
import csv
import torch

import numpy as np
from torch_geometric.data import Data

In [2]:
graph_enc = pd.read_csv("graph_omsk_F_encoded.csv", index_col=[0])

In [3]:
graph = graph_enc.drop(["style_<pandas.io.formats.style.Styler object at 0x000001BA9BC98AC0>"], axis = 1)

In [4]:
def get_index_tuples(df):
    lst = []
    for i in tqdm(range(len(df))):
        tmp = df['adjacent'][i].split(',')
        for edge in tmp:
            lst.append([i, int(edge)])
            
    return lst
lst = get_index_tuples(graph_enc)
lst = pd.DataFrame(lst)
lst

100%|███████████████████████████████████████████████████████████████████████| 231688/231688 [00:06<00:00, 37058.17it/s]


,0,1
0,0,1
1,0,2
2,0,111230
3,0,3
4,0,111231
...,...,...
1149487,231683,231682
1149488,231684,231685
1149489,231685,231684
1149490,231686,231687


In [6]:
graph

,edge_id,length,width,rm_car_speed,type,light_begin,light_end,turn_to,payments_flags,is_city_edge,barrier,turn_restrictions,traverse,passability,pedo_offset,bad_road,def_speed,road_exit,adjacent,road_classes_1,road_classes_2,road_classes_3,road_classes_4,road_classes_5,road_classes_6,road_classes_7,road_classes_8,road_classes_9,sm_classes_0,sm_classes_1,sm_classes_2,sm_classes_3,sm_classes_4,sm_classes_5,sm_classes_6,sm_classes_7,sm_classes_8,sm_classes_9,sm_classes_10,sm_classes_11,sm_classes_12,sm_classes_13,sm_classes_14,sm_classes_15,sm_classes_16,sm_classes_17,sm_classes_18,sm_classes_19,sm_classes_20,sm_classes_21,sm_classes_22,sm_classes_23,sm_classes_24,sm_classes_25,sm_classes_26,sm_classes_27,sm_classes_28,sm_classes_29,sm_classes_30,sm_classes_31,sm_classes_32,sm_classes_33,sm_classes_34,sm_classes_35,sm_classes_36,sm_classes_37,sm_classes_38,sm_classes_39,sm_classes_40,sm_classes_41,sm_classes_42,sm_classes_43,sm_classes_44,sm_classes_45,sm_classes_46,sm_classes_47,sm_classes_48,lanes_0,lanes_1,lanes_2,lanes_3,lanes_4,lanes_5,rm_car_classes_0,rm_car_classes_1,rm_car_classes_2,rm_car_classes_3,rm_car_classes_4,rm_car_classes_5,rm_car_classes_6,rm_car_classes_7,rm_car_classes_8,rm_car_classes_9,rm_car_classes_10,rm_car_classes_11,rm_car_classes_12,rm_car_classes_13,rm_car_classes_14,rm_car_classes_15,rm_car_classes_16,rm_car_classes_17,rm_car_classes_18
0,564384472498178,242,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"1, 2, 111230, 3, 111231",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,564384472498179,242,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"0, 111308, 111309",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,564384472498182,224,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"3, 6, 29, 7, 28",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,564384472498183,224,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"1, 2, 111230, 0, 111231",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,564384472498184,444,4,90,0,0,0,1,0,0,0,0,1,1,1,0,90,0,"5, 16, 17",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231683,212117746323998779,7,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231682,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
231684,212117746366813904,11,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231685,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
231685,212117746366813905,11,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231684,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
231686,212117746395564138,16,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231687,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
edge_index = torch.tensor(np.array(lst).T, dtype=torch.long)

In [62]:
train_mask = []
test_mask = []
for i in range(len(graph)):
    train_mask.append(True)
    test_mask.append(False)
for i in range(int(len(graph)*4/5), len(graph)):
    train_mask[i] = False
    test_mask[i] = True

In [63]:
train_mask = torch.tensor(train_mask, dtype=torch.bool)
test_mask = torch.tensor(test_mask, dtype=torch.bool)

In [56]:
x = torch.tensor(graph_enc.drop(["edge_id", "adjacent"], axis = 1).to_numpy(), dtype=torch.float)
y = torch.tensor(graph_enc["length"].to_numpy(), dtype=torch.float)
data = Data(x=x, edge_index=edge_index, train_mask = train_mask, test_mask = test_mask)

In [57]:
data

Data(edge_index=[2, 1149492], test_mask=[231688], train_mask=[231688], x=[231688, 101])